In [1]:
import sys
from PyQt5.QtWidgets import QMainWindow, QAction, qApp, QApplication, QPlainTextEdit, QToolTip
from PyQt5.QtGui import QIcon,QFont
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QDate, QDateTime, QTime, Qt
import time
import pandas as pd

# imports for graphing
import pyqtgraph as pg
from urllib.request import urlretrieve
import zipfile

#*
#
# Author: John Mulcahy
# Student # 2916973
# Assignment 1 for HCI & GUI development
#*

class Example(QMainWindow):
    
    def __init__(self):
        super().__init__()
        
        self.initUI()
        
        
    def initUI(self):               
        
        exitAct = QAction(QIcon('exit.png'), '&Exit', self)        
        exitAct.triggered.connect(qApp.quit)
        self.statusBar()
        
        self.df = self.download_unzip() # call method to download data and unzip returns a dataframe
        time.sleep(5)
        menubar = self.menuBar()
        fileMenu = menubar.addMenu('&File')
        editMenu = menubar.addMenu('&Edit')
        helpMenu = menubar.addMenu('&Help')
        fileMenu.addAction(exitAct)
        fileMenu.addMenu('New')
        fileMenu.addMenu('Open')
        fileMenu.addMenu('Close')
        
        currencies = ['USD', 'JPY', 'BGN', 'CYP', 'CZK', 'DKK', 'EEK', 'GBP', 'HUF', 'LTL','LVL', 'MTL', 'PLN', 'ROL','RON', 'SEK', 'SIT', 'SKK', 'CHF', 'ISK', 'NOK', 'HRK',	'RUB', 'TRL', 'TRY', 'AUD', 'BRL',
                                              'CAD', 'CNY', 'HKD', 'IDR', 'ILS', 'INR','KRW','MXN', 'MYR', 'NZD', 'PHP', 'SGD', 'THB', 'ZAR' ]
       
       ### initiate elements for gui application ###
        self.lbl = QLabel("Select a currency", self)
        self.gui_graph_label = QLabel("Currency Graph",self)
       
        self.fromComboBox = QComboBox(self)
        self.fromComboBox.addItems(currencies)
        self.toComboBox = QComboBox(self)
        self.toComboBox.addItems(currencies)
        self.fromSpinBox = QDoubleSpinBox(self)
        
        self.fromSpinBox.setRange(0.01, 10000000.00)
        self.fromSpinBox.setValue(1.00)
        self.toLabel = QLabel("1.00", self)
        self.toLabel.setFont(QFont('SansSerif', 10))
        
        self.calMessagelbl = QLabel("Select dates from the calanders",self)
        self.calFrom = QCalendarWidget(self)
        self.calFrom.setMinimumDate(QDate(1900, 1, 1))
        self.calFrom.setMaximumDate(QDate(3000, 1, 1))
        self.calFrom.setGridVisible(True)
        
        self.calTo = QCalendarWidget(self)
        self.calTo.setMinimumDate(QDate(1900, 1, 1))
        self.calTo.setMaximumDate(QDate(3000, 1, 1))
        self.calTo.setGridVisible(True)
        
        self.btn1 = QPushButton("Graph 1", self)
        self.btn1.setToolTip('Graphs the currencies selected and other')
        self.btn2 = QPushButton("Graph 2", self)
        self.btn2.setToolTip('Graphs only the two currencies selected')
        
        self.btn3_clear_graph = QPushButton("Clear Graph", self)
        self.btn3_clear_graph.setToolTip('Clears the Graph')
        
        self.fromDateLabel = QLabel("Start Date: ", self)
        self.fromDateLabel.setFont(QFont('SansSerif', 8))
        
        self.toDateLabel = QLabel("Finish Date: ", self)
        self.toDateLabel.setFont(QFont('SansSerif', 8))
        
        self.graph_button_message = QLabel("Click the buttons to graph the currencies",self)
        self.warning_messagaLabel = QLabel("", self)
        self.graph_button_1 = QLabel("Graph 1 plots the values of the currencies selected and one other",self)
        self.graph_button_2 = QLabel("Graph 2 plots the values of only the currencies selected",self)
        
        
        
    ### positioning of elements in GUI design ###
        self.lbl.move(20,25)
        self.lbl.setFont(QFont('SansSerif', 10))
        self.lbl.adjustSize()
        
        self.gui_graph_label.move(620,20)
        self.gui_graph_label.setFont(QFont('SansSerif', 12))
        self.gui_graph_label.adjustSize()
        
        self.fromComboBox.move(20, 50)
        self.toComboBox.move(130, 50)
        self.fromSpinBox.move(20, 90)
        self.toLabel.move(130,90)
        
        self.calMessagelbl.move(20,150)
        self.calMessagelbl.setFont(QFont('SansSerif', 10))
        self.calMessagelbl.adjustSize()
        
        self.btn1.clicked.connect(self.buttonClickedGraph1)
        self.btn2.clicked.connect(self.buttonClickedGraph2)
        #self.btn1.clicked.connect(self.clear_graphs)
       # self.btn2.clicked.connect(self.clear_graphs)
        self.btn3_clear_graph.clicked.connect(self.clear_graphs)
        
        self.calFrom.move(20,200)
        self.calFrom.resize(240,210)
        self.calFrom.adjustSize()
        
        self.calTo.move(20,430)
        self.calTo.resize(240,210)
        self.calTo.adjustSize()
        
        self.graph_button_message.move(360,560)
        self.graph_button_message.setFont(QFont('SansSerif', 11))
        self.graph_button_message.adjustSize()
        
        self.graph_button_1.move(360,640)
        self.graph_button_1.setFont(QFont('SansSerif', 10))
        self.graph_button_1.adjustSize()
        
        self.graph_button_2.move(360,660)
        self.graph_button_2.setFont(QFont('SansSerif', 10))
        self.graph_button_2.adjustSize()
        
        self.btn1.move(580, 600)
        self.btn2.move(750, 600)
        self.btn3_clear_graph.move(850,600)
        
        self.fromDateLabel.move(20,175)
        self.fromDateLabel.setFont(QFont('SansSerif', 10))
        self.fromDateLabel.adjustSize()
        
        self.toDateLabel.move(20,400)
        self.toDateLabel.setFont(QFont('SansSerif', 10))
        self.toDateLabel.adjustSize()
        
        self.warning_messagaLabel.move(360,600)
        self.warning_messagaLabel.setFont(QFont('SansSerif', 12))
        self.warning_messagaLabel.setStyleSheet("color: #ff0000; color: #ff0000;")
        self.warning_messagaLabel.resize(140,25)
        
        self.setGeometry(200, 30, 1000, 690)
        self.setWindowTitle("Currency Converter") 
        
    ### currency converter update gui when selection has changed ###
        self.fromComboBox.currentIndexChanged.connect(self.updateUi)
        self.toComboBox.currentIndexChanged.connect(self.updateUi)
        self.fromSpinBox.valueChanged.connect(self.updateUi)
       
       
    ### create plot set initial graph for GUI###
        self.plt = pg.PlotWidget(self)
        self.plt.showGrid(x=True, y=True)
        self.plt.addLegend()
        self.l = self.plt.plotItem.legend
        self.l.hide()
        self.plt.setLabel('left', 'Rate')
        self.plt.setLabel('bottom', 'Days')

        self.plt.clear()
    
        date_range = range(0, 10) # should be equal to the number of days you want to have in your x-axis
    
        self.plt.setXRange(0, 10)  # should be as wide as the number of dates we want to show
        self.plt.setYRange(0, 20)  # should be as high as the max exchange

        # rates to be plotted. You will want to get this from your self.data object
        rates1 = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
        rates2 = [0, 1, 2, 4, 12, 14, 16, 17, 14, 22]
        conversion3 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        
        x = range(0, 10)
       # c1 = self.plt.plot(date_range, rates1, pen='b', symbol='x', symbolPen='b', symbolBrush=0.2, name='Cur_2')
       # c2 = self.plt.plot(date_range, rates2, pen='r', symbol='o', symbolPen='r', symbolBrush=0.2, name='Cur_1')
       # c1 = self.plt.plot(date_range, conversion3, pen='g', symbol='+', symbolPen='g', symbolBrush=0.2, name='othr_Cur')
        
       
        self.plt.move(365,50) 
        self.plt.resize(600,500)
        
        self.show()
    ###-------------------------------------------------------###

    def download_unzip(self): # method to download data and unzip returns a dataframe 
    
        url = 'https://www.ecb.europa.eu/stats/eurofxref/eurofxref-hist.zip' # new data source
        file, _ = urlretrieve(url)
        zip_file_object = zipfile.ZipFile(file)
        first_file = zip_file_object.namelist()[0]
        file = zip_file_object.open(first_file)
        data = file
        
        self.df = pd.read_csv(data.name, index_col='Date',  names=['Date','USD', 'JPY', 'BGN', 'CYP', 'CZK', 'DKK', 'EEK', 'GBP', 'HUF', 'LTL','LVL', 'MTL', 'PLN', 'ROL','RON', 'SEK', 'SIT', 'SKK', 'CHF', 'ISK', 'NOK', 'HRK',	'RUB', 'TRL', 'TRY', 'AUD', 'BRL',
                                              'CAD', 'CNY', 'HKD', 'IDR', 'ILS', 'INR','KRW','MXN', 'MYR', 'NZD', 'PHP', 'SGD', 'THB', 'ZAR' ])

   
        return self.df
        
    def updateUi(self): # method for updating the currency converter section in GUI application
        to_currency = self.toComboBox.currentText()
        from_currency = self.fromComboBox.currentText()
          
        amt = self.fromSpinBox.value()
        
        ########## getting information selected ##########
        to_date = self.calTo.selectedDate()
        to_date = to_date.toString("dd-MM-yyyy")
        to_date = to_date.replace('-','/')
        from_date = self.calFrom.selectedDate()
        
        from_date = from_date.toString("dd-MM-yyyy")
        from_date = from_date.replace('-','/')
        ######## show dates selected #################
        
        self.fromDateLabel.setText("Start Date: " + from_date)
        self.fromDateLabel.adjustSize()
        self.toDateLabel.setText("Finish Date: " + to_date)
        self.toDateLabel.adjustSize()
        self.l = self.plt.plotItem.legend
        self.l.hide()
        self.plt.clear() # clear graph
        try:
            currency_1 = self.get_data(to_date,from_date,to_currency,self.df)
            c1 = currency_1[0]
            
            currency_2 = self.get_data(to_date,from_date,self.fromComboBox.currentText(),self.df)
            c2 = currency_2[0]
            to_amount = (c2 / c1) * amt
            
            # update calculated value to label
            self.toLabel.setText("%.02f" % to_amount)  
        except IndexError:
            print("Out of range error")
        
            
    def buttonClickedGraph1(self): # method for graph 1 button plot and get data
        max_total = 0 # create variable for max range
        
        self.plt.clear() # clear graph
        
        checker = False
        self.plt.addLegend()
        self.l = self.plt.plotItem.legend
        # later on, clear all items and re-add
        cc = self.l.layout.count()
        print(cc)
        
        if self.l.layout.count() <= 0:
            print()
        else:
            count = 0
            print("hide")
            self.l.hide()
            count = count + 1
            checker = True
        if checker:
                self.plt.addLegend()    
 
        
        ### getting information selected ###
        to_currency = self.toComboBox.currentText()
        from_currency = self.fromComboBox.currentText()
        to_date = self.calTo.selectedDate()
        to_date = to_date.toString("dd-MM-yyyy")
        to_date = to_date.replace('-','/')
            
        from_date = self.calFrom.selectedDate()
        from_date = from_date.toString("dd-MM-yyyy")
        from_date = from_date.replace('-','/')
        self.fromDateLabel.setText("Start Date: " + from_date)
        self.fromDateLabel.adjustSize()
        self.toDateLabel.setText("Finish Date: " + to_date)
        self.toDateLabel.adjustSize()
        
        try:    
            ### get data ###
            currency_1 = self.get_data(to_date,from_date,to_currency,self.df)
            currency_2 = self.get_data(to_date,from_date,from_currency,self.df)
            currency_other = self.get_data(to_date,from_date,'PLN',self.df)
    
            try:
                ### get the maximum amount range from currencies
                max_cur_1 = max(currency_1)
                max_cur_2 = max(currency_2)
                max_cur_other = max(currency_other)
                max_total = max(max_cur_1,max_cur_2,max_cur_other)
            except ValueError:
                print("No max value")
            
            ### get the total number of days with values returned ###
            days_length_cur1 = len(currency_1)
            days_length_cur2 = len(currency_2)
            days_length_cur3 = len(currency_other)
            
            date_range_c1 = range(0, days_length_cur1) # should be equal to the number of days you want to have in your x-axis
            date_range_c2 = range(0, days_length_cur2)
            date_range_c3 = range(0, days_length_cur3)
        
            self.plt.setXRange(1, days_length_cur1)  # should be as wide as the number of dates we want to show
            self.plt.setYRange(1, max_total)  # should be as high as the max exchange
            
            ### ploting values to the graph ###
            rates_plot_1 = self.plt.plot(date_range_c1, currency_1, pen='b')
            rates_plot_2 = self.plt.plot(date_range_c2, currency_2, pen='r')
            rates_plot_3 = self.plt.plot(date_range_c3, currency_other, pen='g')
            
            
            self.l.addItem(rates_plot_1, from_currency)
            self.l.addItem(rates_plot_2, to_currency)
            self.l.addItem(rates_plot_3, "PLN")
            
            
                        
        except IndexError:
            print("Out of range error")
      
    def buttonClickedGraph2(self): ## method for graph 2 button plot and get data
        max_total = 0 # create variable for max range
        self.plt.clear() # clear graph
        checker = False
        self.plt.addLegend()
        self.l = self.plt.plotItem.legend
        # later on, clear all items and re-add
        
        if self.l.layout.count() <= 0:
            print()
        else:
            count = 0
            print("hide")
            count = count + 1
            checker = True
            self.l.hide()
        if checker:
                self.plt.addLegend()
        
          
        ########## getting information selected ##########
        to_currency = self.toComboBox.currentText()
        from_currency = self.fromComboBox.currentText()
        to_date = self.calTo.selectedDate()
        to_date = to_date.toString("dd-MM-yyyy")
        to_date = to_date.replace('-','/')
            
        from_date = self.calFrom.selectedDate()
        from_date = from_date.toString("dd-MM-yyyy")
        from_date = from_date.replace('-','/')
        self.fromDateLabel.setText("Start Date: " + from_date)
        self.fromDateLabel.adjustSize()
        self.toDateLabel.setText("Finish Date: " + to_date)
        self.toDateLabel.adjustSize()
            
        try:    
            ### get data ###
            currency_1 = self.get_data(to_date,from_date,to_currency,self.df)
            currency_2 = self.get_data(to_date,from_date,from_currency,self.df)
            
        
            try:
                ### get the maximum amount range from currencies
                max_cur_1 = max(currency_1)
                max_cur_2 = max(currency_2)
                max_total = max(max_cur_1,max_cur_2)
            except ValueError:
                print("No max value")
            
            ### get the total number of days with values returned ###
            days_length_cur1 = len(currency_1)
            days_length_cur2 = len(currency_2)
            
            date_range_c1 = range(0, days_length_cur1) # should be equal to the number of days you want to have in your x-axis
            date_range_c2 = range(0, days_length_cur2)
            
            self.plt.setXRange(1, days_length_cur1)  # should be as wide as the number of dates we want to show
            self.plt.setYRange(1, max_total)  # should be as high as the max exchange
            
            rates_plot_1 = self.plt.plot(date_range_c1, currency_1, pen='b')
            rates_plot_2 = self.plt.plot(date_range_c2, currency_2, pen='r')
            
            
            self.l.addItem(rates_plot_1, from_currency)
            self.l.addItem(rates_plot_2, to_currency)
            
        except IndexError:
            print("Out of range error")
       
    def get_data(self, to_date, from_date, to_currency, df2): # searching through the dataframe with specific dates and currencies return a list of values
        list_cur_values = []
        if (to_date in df2.index) & (from_date in df2.index):
            d = df2.loc[to_date: from_date, to_currency]
            self.warning_messagaLabel.setText("") # clear warning message
            for i in d:
              temp = float(i)
              list_cur_values.append(temp) 
        else:
            self.warning_messagaLabel.setText("Dates not available") # set warning message if date values not available
        return list_cur_values
    
    def clear_graphs(self):
        self.plt.plotItem.legend.hide()
       
        self.plt.clear() # clear graph
   
        
if __name__ == '__main__': 
    app = QApplication(sys.argv)
    ex = Example()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\John\Anaconda3\envs\hgp\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
